In [ ]:
!pip install jsonpath_ng

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import sqlite3
import functools as ft
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod
from jsonpath_ng import parse
from enum import Enum
%matplotlib inline

## Data

In [ ]:
original_file = pd.read_excel("musicData_ex.xlsx")
original_file
original_file .to_json("musicData_ex.json")
json_original = original_file .astype(str).to_dict(orient='records')
json_original

[{'Unnamed: 0': '0',
  'track_id': '5SuOikwiRyPMVoIQDJUgSV',
  'artists': 'Gen Hoshino',
  'album_name': 'Comedy',
  'track_name': 'Comedy',
  'popularity': '73',
  'duration_ms': '230666',
  'explicit': 'False',
  'danceability': '0.676',
  'energy': '0.461',
  'key': '1',
  'loudness': '-6.746',
  'mode': '0',
  'speechiness': '0.143',
  'acousticness': '0.0322',
  'instrumentalness': '1.01e-06',
  'liveness': '0.358',
  'valence': '0.715',
  'tempo': '87.917',
  'time_signature': '4',
  'track_genre': 'acoustic'},
 {'Unnamed: 0': '1',
  'track_id': '4qPNDBW1i3p13qLCt0Ki3A',
  'artists': 'Ben Woodward',
  'album_name': 'Ghost (Acoustic)',
  'track_name': 'Ghost - Acoustic',
  'popularity': '55',
  'duration_ms': '149610',
  'explicit': 'False',
  'danceability': '0.42',
  'energy': '0.166',
  'key': '1',
  'loudness': '-17.235',
  'mode': '1',
  'speechiness': '0.0763',
  'acousticness': '0.924',
  'instrumentalness': '5.56e-06',
  'liveness': '0.101',
  'valence': '0.267',
  'tempo'

###  Abstract Base Calss (ABC) for mutual methods

In [ ]:
class Interface(ABC):
    @abstractmethod
    def get_data_by_field(self, field_name):
        """Fetch the data by given feild name """

    @abstractmethod
    def get_data_by_id(self, id):
        """Fetch the data by given ID  """

    @abstractmethod
    def get(self):
        """Fetch all data """

In [ ]:
class get_data(Interface):
    def get_data_by_id(self, id):
        self.id = id
        data = self.get
        for x in data:
            if x.get("id").__str__() == self.id.__str__():
                return x
        return None

    def get_data_by_field(self, field_name):
        data = self.get
        for item in data:
            for key, value in item.items():
                if key == field_name:
                    return item
        return None


### Transform Operations
inherithed from Enum - class that automatic enumrate the variables

In [ ]:
class TransformMask(Enum):
    # add here any masks you want
    CLEAN_STRING = ".strip().lower()"
    CAPITAL_LETTER = ".strip().lower().title()"
    NO_ACT = ""

### Database Class - Define Common Properties for Source, Target, Mapping

In [ ]:
class Database:
    def __init__(self):
        self.db = {
            "source": [],
            "destination": [],
            "transform": [],
            "mapping": []
        }

        self.add_source(1,"artists","$.artists","str")
        self.add_source(2,"track_name","$.track_name","str")
        self.add_source(3,"popularity","$.popularity","str")
        self.add_source(4,"album_name","$.album_name","str")
        self.add_source(5,"track_genre","$.track_genre","str")
        self.add_source(6,"duration_ms","$.duration_ms","str")
        self.add_source(7,"loudness","$.loudness","str")
        self.add_source(8,"speechiness","$.speechiness","str")
        self.add_source(9,"acousticness","$.acousticness","str")
        self.add_source(10,"danceability","$.danceability","str")
        self.add_source(11,"energy","$.energy","str")
        self.add_source(12,"valence","$.valence","str")
        self.add_source(13,"tempo","$.tempo","str")


        self.add_destination(1,"artists","str","n/a")
        self.add_destination(2,"track_name","str","n/a")
        self.add_destination(3,"popularity","int","0")
        self.add_destination(4,"album","str","n/a")
        self.add_destination(5,"genre","str","n/a")
        self.add_destination(6,"duration","int","0")
        self.add_destination(7,"loudness","float","0")
        self.add_destination(8,"speechiness","float","0")
        self.add_destination(9,"acousticness","float","0")
        self.add_destination(10,"danceability","float","0")
        self.add_destination(11,"energy","float","0")
        self.add_destination(12,"valence","float","0")
        self.add_destination(13,"tempo","float","0")



        self.add_transform(1,'CLEAN_STRING')
        self.add_transform(2,'CAPITAL_LETTER')
        self.add_transform(3,'NO_ACT')
        self.add_transform(4,'REPLACE_TO_BOOL')


        self.add_mapping(1,1,1,2,"fact")
        self.add_mapping(2,2,2,2,"fact")
        self.add_mapping(3,3,3,3,"fact")
        self.add_mapping(4,4,4,2,"GeneralData")
        self.add_mapping(5,5,5,2,"GeneralData")
        self.add_mapping(6,6,6,3,"GeneralData")
        self.add_mapping(7,7,7,3,"TechnicalValues")
        self.add_mapping(8,8,8,3,"TechnicalValues")
        self.add_mapping(9,9,9,3,"TechnicalValues")
        self.add_mapping(10,10,10,3,"SubjectiveValues")
        self.add_mapping(11,11,11,3,"SubjectiveValues")
        self.add_mapping(12,12,12,3,"SubjectiveValues")
        self.add_mapping(13,13,13,3,"SubjectiveValues")


    def add_source(self,id,field,map,ty):
      if ty not in ["int","float","str"]:
        print("Error!")
      self.db["source"].append({  "id":id,
                                  "source_field_name":field,
                                  "source_field_mapping":map,
                                  "source_field_type":ty,
                                  "is_required": True,
      })

    def add_destination(self,id,field,ty,default):
        if ty not in ["int","float","str","Bool"]:
          print("Error destination.")
        self.db["destination"].append(
            {         "id": id,
                      "destination_field_name": field,
                      "destination_field_mapping": field,
                      "destination_field_type": ty,
                      "default_value": default,
                  })

    def add_transform(self,id, transform_mask):
        self.db["transform"].append({
                      "id": id,
                      "transform_mask": transform_mask
                  })

    def add_mapping(self,id,mapping_source,mapping_destination,mapping_transform,table):
        self.db["mapping"].append({
                      "id": id,
                      "mapping_source": mapping_source,
                      "mapping_destination": mapping_destination,
                      "mapping_transform": mapping_transform,
                      "mapping_table": table
                  })

    @property
    def get_data_source_target_mapping(self):
        return self.db


### Source class , Target class , Transform Class , Mapping class

In [ ]:
class Source(get_data, Database):
    def __init__(self):
        Database.__init__(self)

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("source")

In [ ]:
class Target(get_data, Database):
    def __init__(self):
        Database.__init__(self)

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("destination")

In [ ]:
class Transform(get_data, Database):

    def __init__(self):
        Database.__init__(self)

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("transform", [])

In [ ]:
class Mappings(get_data, Database):

    def __init__(self):
        Database.__init__(self)

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("mapping")

    def get_data_by_field(self, field_name):
        return None

### Format Class - JSON

Search the source data value inside a JSON file

In [ ]:
class JsonQuery:
    def __init__(self, json_path, json_data):
        self.json_path = json_path
        self.json_data = json_data

    def get(self):
        jsonpath_expression = parse(self.json_path)
        match = jsonpath_expression.find(self.json_data)
        source_data_value = match[0].value
        return source_data_value

### Combine it All - STTM

In [ ]:
class STTM:
    def __init__(self, input_json):
        self.input_json = input_json
        self.mapping_instance = Mappings()
        self.source_instance = Source()
        self.destination_instance = Target()
        self.transform_instance = Transform()
        self.look_up_mask = {i.name: i.value for i in TransformMask}
        self.json_data_transformed = {}
        self.to_table = {}

    def _get_mapping_data(self):
        return self.mapping_instance.get

    def _get_mapping_source_data(self):
        return self.source_instance.get

    def get_transformed_data(self):

        for mappings in self._get_mapping_data():

            """fetch the source mapping """
            mapping_source_id = mappings.get("mapping_source")
            mapping_destination_id = mappings.get("mapping_destination")
            mapping_transform_id = mappings.get("mapping_transform")
            mapping_table = mappings.get("mapping_table")


            mapping_source_data = self.source_instance.get_data_by_id(id=mapping_source_id)
            transform_data = self.transform_instance.get_data_by_id(id=mapping_transform_id)

            """Fetch Source  field Name"""
            source_field_name = mapping_source_data.get("source_field_name")

            """if field given is not present incoming json """
            if source_field_name not in self.input_json.keys():
                if mapping_source_data.get("is_required"):
                    raise Exception(
                        "Alert ! Field {} is not present in JSON please FIX mappings ".format(source_field_name))
                else:
                    pass

            else:
                source_data_value = JsonQuery(
                    json_path=mapping_source_data.get("source_field_mapping"),
                    json_data=self.input_json
                ).get()

                """check the data type for source if matches with what we have """
                if mapping_source_data.get("source_field_type") != type(source_data_value).__name__:
                    if source_data_value is not None:
                        _message = (
                            "Alert ! Source Field :{} Datatype has changed from {} to {} ".format(source_field_name,
                                                                                                  mapping_source_data.get(
                                                                                                      "source_field_type"),
                                                                                                  type(
                                                                                                      source_data_value).__name__))
                        print(_message)
                        raise Exception(_message)

                """Query and fetch the Destination | target """
                destination_mappings_json_object = self.destination_instance.get_data_by_id(
                    id=mappings.get("mapping_destination"))

                destination_field_name = destination_mappings_json_object.get("destination_field_name")
                destination_field_type = destination_mappings_json_object.get("destination_field_type")
                self.to_table[destination_field_name] = mapping_table

                dtypes = [str, float, list, int, set, dict]

                for dtype in dtypes:

                    """Datatype Conversion """
                    if destination_field_type == str(dtype.__name__):

                        """is source is none insert default value"""
                        if source_data_value is None:
                            self.json_data_transformed[destination_field_name] = dtype.__call__(
                                destination_mappings_json_object.get("default_value")
                            )

                        else:
                            """check if you have items to transform"""
                            if transform_data is not None:
                                """ check for invalid mask name """
                                if transform_data.get("transform_mask") not in list(self.look_up_mask.keys()):
                                    raise Exception(
                                        f"Specified Transform {transform_data.get('transform_mask')} is not available please select from following Options :{list(self.look_up_mask.keys())}")
                                else:
                                    mask_apply = self.look_up_mask.get(transform_data.get("transform_mask"))
                                    converted_dtype = dtype.__call__(source_data_value)
                                    mask = f'converted_dtype{mask_apply}'
                                    curated_value = eval(mask)
                                    self.json_data_transformed[destination_field_name] = curated_value

                            else:
                                self.json_data_transformed[destination_field_name] = dtype.__call__(source_data_value)

        return self.json_data_transformed , self.to_table

In [ ]:
transformed_data = []
for item in json_original:
    helper = STTM(input_json=item)
    response , mapping = helper.get_transformed_data()
    transformed_data.append(response)
    print(response)
print(mapping)

{'artists': 'Gen Hoshino', 'track_name': 'Comedy', 'popularity': 73, 'album': 'Comedy', 'genre': 'Acoustic', 'duration': 230666, 'loudness': -6.746, 'speechiness': 0.143, 'acousticness': 0.0322, 'danceability': 0.676, 'energy': 0.461, 'valence': 0.715, 'tempo': 87.917}
{'artists': 'Ben Woodward', 'track_name': 'Ghost - Acoustic', 'popularity': 55, 'album': 'Ghost (Acoustic)', 'genre': 'Acoustic', 'duration': 149610, 'loudness': -17.235, 'speechiness': 0.0763, 'acousticness': 0.924, 'danceability': 0.42, 'energy': 0.166, 'valence': 0.267, 'tempo': 77.489}
{'artists': 'Ingrid Michaelson;Zayn', 'track_name': 'To Begin Again', 'popularity': 57, 'album': 'To Begin Again', 'genre': 'Acoustic', 'duration': 210826, 'loudness': -9.734, 'speechiness': 0.0557, 'acousticness': 0.21, 'danceability': 0.438, 'energy': 0.359, 'valence': 0.12, 'tempo': 76.332}
{'artists': 'Kina Grannis', 'track_name': "Can'T Help Falling In Love", 'popularity': 71, 'album': 'Crazy Rich Asians (Original Motion Picture S

In [ ]:
mapping

{'artists': 'fact',
 'track_name': 'fact',
 'popularity': 'fact',
 'album': 'GeneralData',
 'genre': 'GeneralData',
 'duration': 'GeneralData',
 'loudness': 'TechnicalValues',
 'speechiness': 'TechnicalValues',
 'acousticness': 'TechnicalValues',
 'danceability': 'SubjectiveValues',
 'energy': 'SubjectiveValues',
 'valence': 'SubjectiveValues',
 'tempo': 'SubjectiveValues'}

In [ ]:
df = pd.DataFrame(transformed_data)
df

,artists,track_name,popularity,album,genre,duration,loudness,speechiness,acousticness,danceability,energy,valence,tempo
0,Gen Hoshino,Comedy,73,Comedy,Acoustic,230666,-6.746,0.1430,0.03220,0.676,0.4610,0.715,87.917
1,Ben Woodward,Ghost - Acoustic,55,Ghost (Acoustic),Acoustic,149610,-17.235,0.0763,0.92400,0.420,0.1660,0.267,77.489
2,Ingrid Michaelson;Zayn,To Begin Again,57,To Begin Again,Acoustic,210826,-9.734,0.0557,0.21000,0.438,0.3590,0.120,76.332
3,Kina Grannis,Can'T Help Falling In Love,71,Crazy Rich Asians (Original Motion Picture Sou...,Acoustic,201933,-18.515,0.0363,0.90500,0.266,0.0596,0.143,181.740
4,Chord Overstreet,Hold On,82,Hold On,Acoustic,198853,-9.681,0.0526,0.46900,0.618,0.4430,0.167,119.949
...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,Frank Turner,Haven'T Been Doing So Well,30,Fthc (Deluxe),Acoustic,196813,-3.910,0.1330,0.00012,0.367,0.8920,0.799,81.928
497,The Bridge City Sinners,"The Legend Of Olog-Hai, Pt. 2",30,Unholy Hymns,Acoustic,184217,-6.457,0.0398,0.39500,0.258,0.7290,0.462,116.943
498,Brandi Carlile;Lucius,You And Me On The Rock (Feat. Lucius),63,In These Silent Days,Acoustic,230098,-6.635,0.0330,0.15000,0.568,0.6860,0.725,172.075
499,Get Dead,Glitch,30,Dancing With The Curse,Acoustic,129472,-8.688,0.0739,0.14000,0.571,0.6960,0.662,155.133
